In [ ]:
import pandas as pd
import json
# Mount Google Drive and load the dataset
from google.colab import drive
drive.mount('/content/drive')
file_path="/content/drive/MyDrive/attiki odos/cleaned_accidents_data (1).csv"
df = pd.read_csv(file_path)

# Set display options for better visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

with open("/content/drive/MyDrive/attiki odos/aocenterlines.json", "r", encoding="utf-8") as f:
    centerlines = json.load(f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
coordinates_df = df[['lat','lng']].copy() # Use .copy() to avoid SettingWithCopyWarning

In [ ]:
import math
import numpy as np # Will be useful for some calculations
# --- Helper Functions (Ported from TypeScript) ---

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) * math.sin(delta_phi / 2) + \
        math.cos(phi1) * math.cos(phi2) * \
        math.sin(delta_lambda / 2) * math.sin(delta_lambda / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def closest_point_on_segment(px, py, x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    len2 = dx * dx + dy * dy

    if len2 == 0:
        return {'x': x1, 'y': y1, 't': 0, 'distance_to_segment_point_cartesian': 0}

    t = max(0, min(1, ((px - x1) * dx + (py - y1) * dy) / len2))
    proj_x = x1 + t * dx
    proj_y = y1 + t * dy

    # Cartesian distance, not Haversine. This is consistent with your TSX.
    # For actual geographic distance to this projected point, Haversine should be used outside.
    dist_cartesian = math.sqrt((px - proj_x)**2 + (py - proj_y)**2)

    return {'x': proj_x, 'y': proj_y, 't': t, 'distance_to_segment_point_cartesian': dist_cartesian}

def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    y = math.sin(lon2_rad - lon1_rad) * math.cos(lat2_rad)
    x = math.cos(lat1_rad) * math.sin(lat2_rad) - \
        math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(lon2_rad - lon1_rad)
    theta = math.atan2(y, x)
    bearing = (math.degrees(theta) + 360) % 360
    return bearing

def get_direction_name(bearing):
    directions = ["North", "Northeast", "East", "Southeast", "South", "Southwest", "West", "Northwest"]
    return directions[round(bearing / 45) % 8]

def calculate_curvature(p1, p2, p3): # p1, p2, p3 are dicts with 'lat' and 'lng'
    x1, y1 = p1['lng'], p1['lat']
    x2, y2 = p2['lng'], p2['lat']
    x3, y3 = p3['lng'], p3['lat']

    # Check for collinearity (simplified for Cartesian-like calculations on lat/lon)
    # This is an approximation as done in the TSX. For high accuracy, projection to a metric CRS would be better.
    area_triangle = abs((x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)) / 2)
    if area_triangle < 1e-10: # Threshold for collinearity
        return {'radius': float('inf'), 'curvature': 0, 'angle': 0, 'direction': "straight"}

    # Calculate side lengths using Haversine for geographic accuracy
    a = haversine_distance(y2, x2, y3, x3) # Side p2-p3
    b = haversine_distance(y1, x1, y3, x3) # Side p1-p3
    c = haversine_distance(y1, x1, y2, x2) # Side p1-p2

    if a == 0 or b == 0 or c == 0: # Avoid division by zero or invalid triangles
         return {'radius': float('inf'), 'curvature': 0, 'angle': 0, 'direction': "straight"}

    # Radius of circumcircle R = abc / (4 * Area)
    # Using Heron's formula for area: Area = sqrt(s(s-a)(s-b)(s-c)) where s = (a+b+c)/2
    s = (a + b + c) / 2
    # Ensure argument to sqrt is non-negative due to potential floating point inaccuracies for near-collinear points
    area_heron_sq = s * (s - a) * (s - b) * (s - c)
    if area_heron_sq <= 1e-9: # Effectively zero area or invalid triangle due to precision
        return {'radius': float('inf'), 'curvature': 0, 'angle': 0, 'direction': "straight"}

    area_heron = math.sqrt(area_heron_sq)
    if area_heron == 0: # Avoid division by zero
        return {'radius': float('inf'), 'curvature': 0, 'angle': 0, 'direction': "straight"}

    radius = (a * b * c) / (4 * area_heron)
    curvature_val = 1 / radius if radius != 0 else float('inf')


    # Angle calculation (cosine rule on triangle p1-p2-p3, angle at p2)
    # cos(B) = (a^2 + c^2 - b^2) / (2ac)
    if 2 * a * c == 0: # collinear
        angle_deg = 0
    else:
        cos_angle_at_p2 = (a*a + c*c - b*b) / (2 * a * c)
        cos_angle_at_p2 = max(-1, min(1, cos_angle_at_p2)) # Clamp to avoid domain errors
        angle_at_p2_rad = math.acos(cos_angle_at_p2)
        angle_deg = 180 - math.degrees(angle_at_p2_rad) # Turning angle

    # Determine direction (left/right turn)
    # Using cross-product-like calculation on the (lon, lat) pairs as if they are Cartesian
    # (x2-x1)*(y3-y1) - (y2-y1)*(x3-x1) # for p1,p2,p3 (p2 is the vertex)
    # Vector p2p1: (x1-x2, y1-y2)
    # Vector p2p3: (x3-x2, y3-y2)
    # Cross product Z component: (x1-x2)*(y3-y2) - (y1-y2)*(x3-x2)
    cross_product_z = (x1 - x2) * (y3 - y2) - (y1 - y2) * (x3 - x2)
    direction = "straight"
    if abs(angle_deg) > 0.1: # Only assign direction if there's a notable angle
        direction = "left" if cross_product_z > 0 else "right"


    return {'radius': radius, 'curvature': curvature_val, 'angle': angle_deg, 'direction': direction}

def categorize_curvature(curvature_val, angle):
    if curvature_val == 0 or angle < 2: # Consistent with TSX
        return "Straight"
    elif curvature_val > 0.001:  # radius < 1000m
        return "Very Sharp Curve"
    elif curvature_val > 0.0005: # radius < 2000m
        return "Sharp Curve"
    elif curvature_val > 0.00025: # radius < 4000m
        return "Moderate Curve"
    elif curvature_val > 0.0001:  # radius < 10000m
        return "Gentle Curve"
    else:
        return "Very Gentle Curve"

def find_next_curve(closest_p_dict, segment_info_dict, feature_dict, geo_data_dict):
    coords = feature_dict['geometry']['coordinates']
    current_index = segment_info_dict['segment'] # This is segment index, so point index is current_index or current_index + 1

    # We need to find the point on the linestring that is closest_p_dict
    # and then look forward from there.
    # The 't' parameter tells us where on the segment (current_index to current_index+1) our closest_p_dict lies.
    # If t is close to 1, we should start looking for curves from the next segment.

    start_node_index = current_index + 1 # The end node of the current segment is a good starting point for looking ahead
                                         # or if t is very small, even current_index could be the middle of a curve.
                                         # For simplicity, let's check starting from the segment after closest_p_dict's segment.

    min_dist_to_curve = float('inf')
    next_curve_details = None
    next_curve_coords_dict = None

    # Iterate through subsequent points in the current feature to find curves
    for i in range(start_node_index, len(coords) - 1):
        if i == 0: continue # Need at least three points (p_i-1, p_i, p_i+1)

        p1 = {'lat': coords[i-1][1], 'lng': coords[i-1][0]}
        p2 = {'lat': coords[i][1], 'lng': coords[i][0]} # Curve is evaluated at p2
        p3 = {'lat': coords[i+1][1], 'lng': coords[i+1][0]}

        curve_at_p2 = calculate_curvature(p1, p2, p3)

        # Define "significant curve" (consistent with TSX: angle > 10 OR radius < 100 (curvature > 0.01))
        # Note: Your TSX says `curvature > 100`, which implies very small radius. Assuming that was a typo and meant radius < 100 or curvature > 0.01
        is_significant_curve = curve_at_p2['angle'] > 10 or (curve_at_p2['radius'] < 100 and curve_at_p2['radius'] > 0)

        if is_significant_curve:
            dist_to_p2 = haversine_distance(closest_p_dict['lat'], closest_p_dict['lng'], p2['lat'], p2['lng'])

            if dist_to_p2 < min_dist_to_curve: # Found a closer significant curve
                min_dist_to_curve = dist_to_p2
                next_curve_details = curve_at_p2
                next_curve_coords_dict = p2

            if dist_to_p2 > 2000: # Only look 2km ahead (as in TSX)
                break

    if next_curve_details and next_curve_coords_dict:
        return {
            'distance': min_dist_to_curve,
            'info': next_curve_details,
            'coordinates': next_curve_coords_dict
        }
    return None

def analyze_road_turning_profile(road_name_base, geo_data_dict):
    features_for_road = [
        f for f in geo_data_dict['features']
        if f['properties']['NAME'] == road_name_base or f['properties']['NAME'].startswith(road_name_base + '-')
    ]

    left_turns = 0
    right_turns = 0
    total_curves = 0
    total_angle_sum = 0

    for feature in features_for_road:
        coords = feature['geometry']['coordinates']
        if len(coords) >= 3:
            for i in range(1, len(coords) - 1):
                p1 = {'lat': coords[i-1][1], 'lng': coords[i-1][0]}
                p2 = {'lat': coords[i][1], 'lng': coords[i][0]}
                p3 = {'lat': coords[i+1][1], 'lng': coords[i+1][0]}

                result = calculate_curvature(p1, p2, p3)

                if result['angle'] > 5: # Significant turn threshold
                    total_curves += 1
                    total_angle_sum += result['angle']
                    if result['direction'] == "left":
                        left_turns += 1
                    elif result['direction'] == "right":
                        right_turns += 1

    return {
        'totalCurves': total_curves,
        'leftTurns': left_turns,
        'rightTurns': right_turns,
        'averageAngle': total_angle_sum / total_curves if total_curves > 0 else 0
    }


# --- Main Analysis Function (Ported and adapted from findClosestRoadPoint) ---
def analyze_point_features(point_lat, point_lng, geo_data):
    closest_p_final = None
    min_dist_final = float('inf')
    segment_info_final = None
    closest_feature_final = None

    # Iterate through all features (road sections)
    for feature in geo_data['features']:
        road_name = feature['properties']['NAME']
        coordinates = feature['geometry']['coordinates']

        # Iterate through all line segments in the current feature
        for i in range(len(coordinates) - 1):
            lon1, lat1 = coordinates[i]
            lon2, lat2 = coordinates[i+1]

            # Find closest point on this segment (lon, lat order for calc, but store as lat, lon)
            # closest_point_on_segment expects (px, py, x1, y1, x2, y2)
            # here, px=point_lng, py=point_lat, x1=lon1, y1=lat1, x2=lon2, y2=lat2
            segment_projection = closest_point_on_segment(point_lng, point_lat, lon1, lat1, lon2, lat2)

            # Calculate actual geodesic distance from original point to the projected point on segment
            geodesic_dist = haversine_distance(point_lat, point_lng, segment_projection['y'], segment_projection['x'])

            if geodesic_dist < min_dist_final:
                min_dist_final = geodesic_dist
                closest_p_final = {'lat': segment_projection['y'], 'lng': segment_projection['x']}
                segment_info_final = {
                    'segment': i, # Store segment index
                    'roadName': road_name,
                    't': segment_projection['t'], # Proportion along the segment
                    'startPoint': {'lat': lat1, 'lng': lon1},
                    'endPoint': {'lat': lat2, 'lng': lon2}
                }
                closest_feature_final = feature

    analysis_result = {
        'closest_point_lat': None, 'closest_point_lng': None, 'min_distance_m': min_dist_final,
        'on_road_threshold_20m': None, 'road_name': None, 'segment_idx': None,
        't_on_segment': None, 'segment_start_lat': None, 'segment_start_lng': None,
        'segment_end_lat': None, 'segment_end_lng': None, 'segment_length_m': None,
        'bearing_deg': None, 'direction_name': None, 'road_context': None,
        'curvature_radius_m': None, 'curvature_val': None, 'curve_angle_deg': None,
        'curve_direction': None, 'curve_category': None,
        'next_curve_dist_m': None, 'next_curve_radius_m': None, 'next_curve_angle_deg': None,
        'next_curve_direction': None, 'next_curve_category': None, 'next_curve_lat': None, 'next_curve_lng': None,
        'profile_total_curves': None, 'profile_left_turns': None, 'profile_right_turns': None, 'profile_avg_angle': None
    }

    if closest_p_final and segment_info_final and closest_feature_final:
        analysis_result['closest_point_lat'] = closest_p_final['lat']
        analysis_result['closest_point_lng'] = closest_p_final['lng']
        analysis_result['on_road_threshold_20m'] = min_dist_final <= 20
        analysis_result['road_name'] = segment_info_final['roadName']
        analysis_result['segment_idx'] = segment_info_final['segment']
        analysis_result['t_on_segment'] = segment_info_final['t']
        analysis_result['segment_start_lat'] = segment_info_final['startPoint']['lat']
        analysis_result['segment_start_lng'] = segment_info_final['startPoint']['lng']
        analysis_result['segment_end_lat'] = segment_info_final['endPoint']['lat']
        analysis_result['segment_end_lng'] = segment_info_final['endPoint']['lng']

        seg_len = haversine_distance(
            segment_info_final['startPoint']['lat'], segment_info_final['startPoint']['lng'],
            segment_info_final['endPoint']['lat'], segment_info_final['endPoint']['lng']
        )
        analysis_result['segment_length_m'] = seg_len

        bearing = calculate_bearing(
            segment_info_final['startPoint']['lat'], segment_info_final['startPoint']['lng'],
            segment_info_final['endPoint']['lat'], segment_info_final['endPoint']['lng']
        )
        analysis_result['bearing_deg'] = bearing
        analysis_result['direction_name'] = get_direction_name(bearing)

        analysis_result['road_context'] = "Junction/Ramp" if '-' in segment_info_final['roadName'] else "Straight Segment" # Simplified

        # Curvature at the closest point on the segment
        # To evaluate curvature AT the closest point, we ideally need points on the road before and after it.
        # The current segment_info_final refers to the segment index `i` for points `coords[i]` and `coords[i+1]`.
        # The closest_p_final lies ON this segment.
        # For curvature, we need 3 points. Let's use the start (coords[i]), end (coords[i+1]) of the segment,
        # and either coords[i-1] or coords[i+2] if available. This is an approximation.
        # A more robust approach would be to find the *actual* point on the linestring that IS closest_p_final
        # (or very near it, like segment_info_final.startPoint if t=0, or .endPoint if t=1)
        # and then use its neighbors on the LineString.

        coords = closest_feature_final['geometry']['coordinates']
        idx_on_linestring = -1
        # If closest_p_final is very close to startPoint of segment
        if segment_info_final['t'] < 0.01 and segment_info_final['segment'] > 0:
             idx_on_linestring = segment_info_final['segment'] # Use start node of current segment
        # If closest_p_final is very close to endPoint of segment
        elif segment_info_final['t'] > 0.99 and segment_info_final['segment'] < len(coords) - 2:
            idx_on_linestring = segment_info_final['segment'] + 1 # Use end node of current segment
        # If it's in the middle, or at the very start/end of a feature's linestring, curvature calc is tricky without more context.
        # For simplicity, if it's in the middle of a segment, we might not have a clear vertex for curvature from original points.
        # The TSX seems to calculate it based on segment_info.segment which is an *index*.
        # Let's use the end point of the current segment as p2 for curvature calculation.

        current_seg_idx = segment_info_final['segment']
        if current_seg_idx > 0 and current_seg_idx < len(coords) -1: # Need p(i-1), p(i), p(i+1)
            # Let's use the segment's end point (coords[current_seg_idx+1]) as the vertex p2
            # This means p1 = coords[current_seg_idx], p2 = coords[current_seg_idx+1], p3 = coords[current_seg_idx+2]
            # if current_seg_idx + 2 < len(coords):
            #     p1_curv = {'lat': coords[current_seg_idx][1], 'lng': coords[current_seg_idx][0]}
            #     p2_curv = {'lat': coords[current_seg_idx+1][1], 'lng': coords[current_seg_idx+1][0]} # Vertex
            #     p3_curv = {'lat': coords[current_seg_idx+2][1], 'lng': coords[current_seg_idx+2][0]}
            # else: # Not enough points, try using the start point as vertex
            if current_seg_idx < len(coords) -1 and current_seg_idx > 0: # Check if we can use start_point as p2
                p1_curv = {'lat': coords[current_seg_idx-1][1], 'lng': coords[current_seg_idx-1][0]}
                p2_curv = {'lat': coords[current_seg_idx][1], 'lng': coords[current_seg_idx][0]} # Vertex (start of current segment)
                p3_curv = {'lat': coords[current_seg_idx+1][1], 'lng': coords[current_seg_idx+1][0]}

                curv_info = calculate_curvature(p1_curv, p2_curv, p3_curv)
                analysis_result['curvature_radius_m'] = curv_info['radius']
                analysis_result['curvature_val'] = curv_info['curvature']
                analysis_result['curve_angle_deg'] = curv_info['angle']
                analysis_result['curve_direction'] = curv_info['direction']
                analysis_result['curve_category'] = categorize_curvature(curv_info['curvature'], curv_info['angle'])
                if curv_info['curvature'] > 0 and analysis_result['road_context'] == "Straight Segment":
                    analysis_result['road_context'] = "Curved Segment"


        # Next Curve (using the actual closest point on road, segment info, and the feature it belongs to)
        next_curve_data = find_next_curve(closest_p_final, segment_info_final, closest_feature_final, geo_data)
        if next_curve_data:
            analysis_result['next_curve_dist_m'] = next_curve_data['distance']
            analysis_result['next_curve_radius_m'] = next_curve_data['info']['radius']
            analysis_result['next_curve_angle_deg'] = next_curve_data['info']['angle']
            analysis_result['next_curve_direction'] = next_curve_data['info']['direction']
            analysis_result['next_curve_category'] = categorize_curvature(next_curve_data['info']['curvature'], next_curve_data['info']['angle'])
            analysis_result['next_curve_lat'] = next_curve_data['coordinates']['lat']
            analysis_result['next_curve_lng'] = next_curve_data['coordinates']['lng']

        # Road Profile
        road_name_base = segment_info_final['roadName'].split('-')[0]
        profile = analyze_road_turning_profile(road_name_base, geo_data)
        analysis_result['profile_total_curves'] = profile['totalCurves']
        analysis_result['profile_left_turns'] = profile['leftTurns']
        analysis_result['profile_right_turns'] = profile['rightTurns']
        analysis_result['profile_avg_angle'] = profile['averageAngle']

        # Nearby junctions and roads (simplified for now - can be expanded as in TSX)
        # This part would involve iterating over all features again for each point, which can be slow.
        # For a large dataset, optimizing this (e.g. spatial indexing) would be crucial.
        # For now, a simplified version or placeholder:
        analysis_result['nearby_junctions_count_500m'] = 0 # Placeholder
        analysis_result['nearby_roads_count_100m'] = 0 # Placeholder


    return pd.Series(analysis_result)

# --- Apply to DataFrame ---
# This might take a while for a large DataFrame
print("Starting analysis for each coordinate point...")
extracted_features_df = coordinates_df.apply(
    lambda row: analyze_point_features(row['lat'], row['lng'], centerlines),
    axis=1
)
print("Analysis complete.")

# Merge the new features back into the original DataFrame
df_augmented = pd.concat([df, extracted_features_df], axis=1)

print("\nAugmented DataFrame head:")
print(df_augmented.head())

print("\nInformation about a sample point (first row):")
print(df_augmented.iloc[0])

Starting analysis for each coordinate point...
Analysis complete.

Augmented DataFrame head:
   α/α   ΗΜ/ΝΙΑ_ΚΑΤΑΧΩΡΗΣΗΣ          ΑΝΑΦΟΡΑ       ΗΜ/ΝΙΑ_ΕΝΑΡΞΗΣ  \
0    1  2014-01-01 00:00:00         Δ.Π.Λ.Υ.  2014-01-01 16:50:00   
1    2  2014-01-11 00:00:00         Δ.Π.Λ.Υ.  2014-01-11 05:41:00   
2    3  2014-01-14 00:00:00  ΕΛ.ΕΣ.Σ. ΔΥΤΙΚΑ  2014-01-14 14:30:00   
3    4  2014-01-19 00:00:00         Δ.Π.Λ.Υ.  2014-01-19 14:34:00   
4    5  2014-01-28 00:00:00  ΕΛ.ΕΣ.Σ. ΔΥΤΙΚΑ  2014-01-28 16:41:00   

              ΗΜ.ΛΗΞΗΣ ΑΥΤ/ΔΡΟΜΟΣ  ΚΟΜΒΟΣ ΣΤΑΘΜ.ΔΙΟΔΙΩΝ ΚΛΑΔΟΣ  ΧΙΛ.ΘΕΣΗ  \
0  2014-01-01 17:33:00          K     2.0           NaN      1       0.0   
1  2014-01-11 07:27:00          P     NaN           NaN    NaN       6.1   
2  2014-01-14 15:36:00          E     1.0           NaN    A-1       0.0   
3  2014-01-19 14:54:00          K     NaN           NaN    NaN       3.8   
4  2014-01-28 17:46:00          A     NaN           NaN    NaN      22.8   

      ΣΗΡΑΓΓΑ                    ΥΠ

In [ ]:
df_augmented

,α/α,ΗΜ/ΝΙΑ_ΚΑΤΑΧΩΡΗΣΗΣ,ΑΝΑΦΟΡΑ,ΗΜ/ΝΙΑ_ΕΝΑΡΞΗΣ,ΗΜ.ΛΗΞΗΣ,ΑΥΤ/ΔΡΟΜΟΣ,ΚΟΜΒΟΣ,ΣΤΑΘΜ.ΔΙΟΔΙΩΝ,ΚΛΑΔΟΣ,ΧΙΛ.ΘΕΣΗ,ΣΗΡΑΓΓΑ,ΥΠΟΚΑΤΗΓΟΡΙΑ,ΑΠΟΤΕΛΕΣΜΑ,ΝΕΚΡΟΙ,ΤΡΑΥΜΑΤΙΕΣ,ΘΕΣΗ_ΕΡΓΟΥ,involved_vehicle_1,involved_vehicle_2,involved_vehicle_3,involved_vehicle_4,involved_vehicle_5,involved_vehicle_6,involved_vehicle_7,ΚΑΙΡ._ΣΥΝΘ.,ΣΥΝΘ._ΟΔ.,ΚΑΤ._ΟΔ.,ΤΕΧΝ._ΦΩΤ.,accident_type,lat,lng,num_involved_vehicles,has_ΙΧΕ,has_ΚΡΑΤΙΚΟ ΟΧΗΜΑ,has_ΔΙΚΥΚΛΟ,has_ΦΟΡΤΗΓΟ,has_ΤΑΞΙ,has_ΛΕΩΦΟΡΕΙΟ,has_Μ.Ε.,temperature_2m,relative_humidity_2m,pressure_msl,wind_speed_10m,wind_direction_10m,wind_gusts_10m,precipitation,weather_code,is_day,weather_description,temperature_category,wind_speed_category,has_precipitation,precipitation_intensity,is_precipitating,precip_at_night,precip_during_day,strong_wind_and_precip,is_foggy,foggy_at_night,is_night_openmeteo,night_lighting_issue_suspected,effective_lighting_condition,year,month,day,hour,minute,day_of_week,is_weekend,is_rush_hour,time_period,season,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos,day_of_month_sin,day_of_month_cos,duration_minutes,duration_hours,duration_category,acc_id,acc_type,original_lat,original_lng,snapped_lat,snapped_lng,closest_toll_euclidean_km,closest_toll_euclidean_name,closest_intersection_euclidean_km,closest_intersection_euclidean_name,recorded_toll_dist_km,recorded_toll_name,recorded_intersection_dist_km,recorded_intersection_name,closest_toll_network_km,closest_toll_network_name,closest_intersection_network_km,closest_intersection_network_name,dist_to_entry_node_km,is_in_tunnel,involves_heavy_vehicle,is_adverse_condition,vehicle_type_count,is_weekend_rush,hour_group,temp_factor,precip_factor,wind_factor,weather_severity,weather_severity_x_num_involved_vehicles,is_covid_year,rain_intensity,is_rainy,is_heavy_rain,wind_level,is_windy,is_strong_wind,temperature_range,is_freezing,is_hot,humidity_level,is_high_humidity,weather_risk_score,weather_risk_level,weather_risk_factor,road_risk_factor,weather_road_interaction,is_rain_on_wet_road,is_snow_or_ice,temp_change_prev,is_rapid_temp_change,hour_of_day,is_early_morning,is_morning_rush,is_midday,is_afternoon,is_evening_rush,is_evening,is_night,is_holiday,is_day_before_holiday,is_day_after_holiday,month_progress,month_part,week_part,is_monday,is_friday,is_saturday,is_sunday,is_weekend_plus,time_since_previous_accident_hours,accidents_past_3h,accidents_past_6h,accidents_past_12h,accidents_past_24h,hour_sin_24,hour_cos_24,hour_sin_12,hour_cos_12,hour_sin_8,hour_cos_8,dow_sin_7,dow_cos_7,dow_sin_5,dow_cos_5,years_since_min,is_rush_hour_enhanced,location_uncertain,road_latitude,road_longitude,road_distance_to_road,road_lanes,road_maxspeed,road_surface,road_smoothness,road_highway,road_bridge,road_tunnel,road_destination,road_tunnel:name,closest_point_lat,closest_point_lng,min_distance_m,on_road_threshold_20m,road_name,segment_idx,t_on_segment,segment_start_lat,segment_start_lng,segment_end_lat,segment_end_lng,segment_length_m,bearing_deg,direction_name,road_context,curvature_radius_m,curvature_val,curve_angle_deg,curve_direction,curve_category,next_curve_dist_m,next_curve_radius_m,next_curve_angle_deg,next_curve_direction,next_curve_category,next_curve_lat,next_curve_lng,profile_total_curves,profile_left_turns,profile_right_turns,profile_avg_angle,nearby_junctions_count_500m,nearby_roads_count_100m
0,1,2014-01-01 00:00:00,Δ.Π.Λ.Υ.,2014-01-01 16:50:00,2014-01-01 17:33:00,K,2.0,NaN,1,0.0,no_ΣΗΡΑΓΓΑ,4D.ΣΤ - ΣΕ ΟΧΗΜΑ ΠΟΥ ΣΤΑΘΜΕΥΕΙ,ΤΡΑΥΜΑΤΙΕΣ,0,1,ΟΧΙ,ΙΧΕ,ΙΧΕ,NaN,NaN,NaN,NaN,NaN,ΚΑΛΟΚΑΙΡΙΑ,ΥΓΡΟ - ΒΡΕΓΜΕΝΟ,ΚΑΝΟΝΙΚΗ,NaN,injury,37.975077,23.795987,2,True,False,False,False,False,False,False,9.5,80,1019.8,11.4,11,22.0,0.3,51,1,Drizzle: Light intensity,Cold,Light Breeze,True,Light Precipitation,True,False,True,False,False,False,False,False,Daylight - Assumed Sufficient (No Manual Entry),2014,1,1,16,50,2,0,1,Afternoon,Winter,-0.866025,-0.500000,0.974928,-0.222521,5.000000e-01,0.866025,0.201299,0.979530,43.0,0.716667,30-60 min,0.0,injury,37.975077,23.7959

In [ ]:
# Define the path in Google Drive
drive_save_path = "/content/drive/MyDrive/attiki odos/augmented_accidents_data_final.csv"

# Save the DataFrame to CSV in Google Drive
df_augmented.to_csv(drive_save_path, index=False)

print(f"DataFrame saved to Google Drive at: {drive_save_path}")

DataFrame saved to Google Drive at: /content/drive/MyDrive/attiki odos/augmented_accidents_data_final.csv


In [ ]:
from google.colab import files

# Define the filename for download
download_filename = "augmented_accidents_data_final.csv"

# Save the DataFrame to a temporary CSV file in the Colab environment for download
df_augmented.to_csv(download_filename, index=False)

# Download the file
files.download(download_filename)

print(f"DataFrame prepared for download as: {download_filename}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame prepared for download as: augmented_accidents_data_final.csv
